In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
# from pyspark.sql.types import StructType, StructField, FloatType

In [2]:
import pandas as pd
import numpy as np

In [3]:
app_name = "My PySpark App1"
master_url = "spark://spark-master:7077"
# local_ip="127.0.0.1"

# # spark.local.ip = helps when multiple network interfaces are present. 
# #                  The driver must be in the same network as the master and slaves
# # spark.driver.host =  same as above. This duality might disappear in a future version 
conf = SparkConf().setAppName(app_name) \
                  .setMaster(master_url) 

In [4]:
sc = SparkContext(conf=conf)

In [5]:
spark = SparkSession.builder \
                         .config(conf=conf) \
                         .getOrCreate()

In [6]:
def mod(x):
#     import numpy as np
    return (x, np.mod(x, 2))

rdd = sc.parallelize(range(1000)).map(mod).take(10)
print(rdd)

[(0, 0), (1, 1), (2, 0), (3, 1), (4, 0), (5, 1), (6, 0), (7, 1), (8, 0), (9, 1)]


In [7]:
num_elements=100000

mu, sigma = 2, 0.5
v = np.random.normal(mu, sigma, num_elements)
rdd1 = sc.parallelize(v)    
def mult(x): return x * np.array([2])
rdd2 = rdd1.map(mult).map(lambda x: (float(x),))    

schema = StructType([StructField("value", FloatType(), True)])
    

df1 = spark.createDataFrame(rdd2, schema)
df1.registerTempTable("test")  

def square(x): return x ** 2

spark.udf.register("squared", square)
df2 = spark.sql("SELECT squared(value) squared FROM test")    
    
df= df2.toPandas()

print(f"Got Pandas dataframe with {df.size} elements")
print(df.head(10))

Got Pandas dataframe with 100000 elements
              squared
0   4.517446045852523
1  15.732332013358302
2  15.746817237579535
3  14.863771555570565
4  25.046675513280206
5  25.663495878697177
6  15.379581295365142
7   13.08941529270038
8  21.426366942065215
9     26.446399128856
